In [56]:
from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [57]:
import struct
from typing import Tuple
from bitarray import bitarray
from src.compressors.compressor import Compressor
import numpy as np
print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=True)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
# Main.include("src/compressors/fp8.jl")
# Main.include("E:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\LinLog_Quantization\LinLogQuantization.jl")
Main.include("E:/MS_Thesis/Hierarchical_quantization/FedPAQ-MNIST-implemenation-main/LinLog_Quantization/LinLogQuantization.jl")
Main.eval('using .LinLogQuantization')  # Use the module
print("Finished starting up Julia.")


Starting up Julia.
Finished starting up Julia.


In [58]:

Main.eval('include("E:/MS_Thesis/Hierarchical_quantization/FedPAQ-MNIST-implemenation-main/LinLog_Quantization/LinLogQuantization.jl")')
Main.eval('using .LinLogQuantization') 
arr = list(np.random.rand(1,16).flatten())
arr = np.array(arr, dtype = np.float32)
LLquant=Main.LinLogQuantization.LinQuant8Array(arr)
print("Before Quantized array:", LLquant)
LLquant = np.array(LLquant, dtype = np.uint8)
# type(LLquant)

print("Original array:", arr)
print("Arr size:", sys.getsizeof(arr))
print("LLquant size:", sys.getsizeof(LLquant))
print("Quantized array:", LLquant)

Before Quantized array: [150, 180, 10, 24, 19, 132, 157, 194, 110, 252, 255, 65, 0, 123, 240, 100]
Original array: [0.5608718  0.66554254 0.07572268 0.12403987 0.10915419 0.50017035
 0.5847847  0.71386635 0.4236727  0.91628206 0.9258732  0.26889813
 0.04240571 0.468164   0.8752635  0.3900149 ]
Arr size: 168
LLquant size: 120
Quantized array: [150 180  10  24  19 132 157 194 110 252 255  65   0 123 240 100]


In [59]:
arr = np.random.rand(1, 16).flatten().astype(np.float32)

print("Original array:", arr)
# Ensure the array is converted to a Julia array
julia_arr = Main.eval("Array{Float32}")(arr.tolist())
LLquant = Main.eval("LinLogQuantization.LinQuant8Array")(julia_arr)
print('decode', julia_arr)
print("Quantized array:", LLquant)

Original array: [0.94599295 0.43729788 0.87890714 0.8318864  0.05353928 0.75466585
 0.13506755 0.21353501 0.14553858 0.6138739  0.29984212 0.3403103
 0.6379253  0.08151774 0.9205873  0.39289764]
decode [0.94599295 0.43729788 0.87890714 0.8318864  0.05353928 0.75466585
 0.13506755 0.21353501 0.14553858 0.6138739  0.29984212 0.3403103
 0.6379253  0.08151774 0.9205873  0.39289764]
Quantized array: [255, 110, 236, 222, 0, 200, 23, 46, 26, 160, 70, 82, 167, 8, 248, 97]


In [25]:
# Prepare data
arr = np.random.rand(10).astype(np.float32)  # Example data
print("Original array:", arr)

# Quantize using Julia
try:
    LLquant = Main.LinQuant8Array(arr)
    # Access the UInt8 array, min, and max from LinQuantArray
    print("Type of LLquant: ", type(LLquant))
    # quantized_data = LLquant.A
    # quant_min = LLquant.min
    # quant_max = LLquant.max
    # print("Quantized array data (sample):", quantized_data[:10])
    # print("Min value:", quant_min, "Max value:", quant_max)

    # Optional: Convert back to floating-point array to check correctness
    dequantized = np.array(Main.Array(float, LLquant))
    print("Dequantized array (sample):", dequantized[:10])

except Exception as e:
    print("Error during quantization:", str(e))

Original array: [0.12619264 0.90614927 0.5356075  0.2161181  0.5300986  0.71093106
 0.21429557 0.06450099 0.91872215 0.15415204]
Type of LLquant:  <class 'list'>
Error during quantization: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching Array(::PyObject, ::Vector{Int64})
Stacktrace:
 [1] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::@Kwargs{})
   @ Base .\essentials.jl:892
 [2] invokelatest(::Any, ::Any, ::Vararg{Any})
   @ Base .\essentials.jl:889
 [3] _pyjlwrap_call(f::Type, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Asus\.julia\packages\PyCall\1gn3u\src\callback.jl:28
 [4] pyjlwrap_call(self_::Ptr{PyCall.PyObject_struct}, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Asus\.julia\packages\PyCall\1gn3u\src\callback.jl:44>


In [26]:
len(LLquant)

10

In [27]:
arr = np.random.rand(100, 100).astype(np.float32)  # Example data
print("Original array:", arr)

# Quantize using Julia
try:
    LLquant_list = Main.LinQuant8Array(arr)
    print("Type of LLquant:", type(LLquant_list))  # Should print <class 'list'>

    # Access elements if it's indeed a list of LinQuantArray
    for idx, LLquant in enumerate(LLquant_list):
        quantized_data = LLquant.A
        quant_min = LLquant.min
        quant_max = LLquant.max
        print(f"Quantized array data (sample) for element {idx}:", quantized_data[:10])
        print("Min value:", quant_min, "Max value:", quant_max)

        # Optional: Convert back to floating-point array to check correctness
        dequantized = np.array(Main.Array(float, LLquant))
        print(f"Dequantized array (sample) for element {idx}:", dequantized[:10])

except Exception as e:
    print("Error during quantization:", str(e))

Original array: [[0.5513071  0.7800149  0.391558   ... 0.5638647  0.40836063 0.37028798]
 [0.32043433 0.04923544 0.460503   ... 0.46973234 0.61585003 0.7816345 ]
 [0.21508895 0.02305379 0.13235116 ... 0.10851511 0.67713124 0.9373543 ]
 ...
 [0.4302368  0.628929   0.73877317 ... 0.13640527 0.5338045  0.18064994]
 [0.6407786  0.75409865 0.2957243  ... 0.5880993  0.46677697 0.09806935]
 [0.4912676  0.5642917  0.95530754 ... 0.03661932 0.4552135  0.15295082]]
Type of LLquant: <class 'list'>
Error during quantization: 'list' object has no attribute 'A'


In [28]:
len(LLquant_list)

100

In [29]:
    # export  LinQuantArray, LogQuantArray,
    # LinQuant8Array, LinQuant16Array, LinQuant24Array, LinQuant32Array,
    # LogQuant8Array, LogQuant16Array, LogQuant24Array, LogQuant32Array,
    # minpos

In [30]:
from abc import ABC, abstractmethod
import numpy as np

class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass

# With Class

In [31]:
import numpy as np
from src.compressors.compressor import Compressor
print("Initializing Julia integration.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
# Main.include("E:/MS_Thesis/Hierarchical_quantization/FedPAQ-MNIST-implemenation-main/src/compressors/linquantarrays.jl")
# Main.include("E:/MS_Thesis/Hierarchical_quantization/FedPAQ-MNIST-implemenation-main/LinLog_Quantization/LinLogQuantization.jl")
Main.include("E:/MS_Thesis/Hierarchical_quantization/FedPAQ-MNIST-implemenation-main/src/compressors/linquantarrays.jl")

print("Julia module loaded successfully.")

Initializing Julia integration.
Julia module loaded successfully.


In [39]:


class LinQuantArrays(Compressor):
    def __init__(self):
        pass

    def encode(self, array: np.ndarray, dtype: np.dtype = np.uint8) -> bytes:
        """Quantizes the numpy array using the specified unsigned integer dtype and returns the encoded bytes."""
        assert len(array.shape) == 1  # Ensure it's a 1D array
        assert array.dtype == np.float32 or array.dtype == np.float64, "Array must be of type float32 or float64"
        
        # Mapping numpy dtype to Julia UInt type
        julia_dtype = {np.uint8: 'UInt8', np.uint16: 'UInt16', np.uint32: 'UInt32', np.uint64: 'UInt64'}[dtype]
        julia_type = Main.eval(julia_dtype)
        # Calling the Julia function to perform quantization
        result = Main.LinQuantization(julia_type, array)
        return result, bytes(result)  # Access the UInt array and convert to bytes


    # def encode(self, array: np.ndarray, dtype: np.dtype = np.uint8):
    #     """Quantizes the numpy array using the specified unsigned integer dtype and returns the encoded bytes along with min and max."""
    #     assert len(array.shape) == 1  # Ensure it's a 1D array
    #     assert array.dtype in [np.float32, np.float64], "Array must be of type float32 or float64"
        
    #     # Mapping numpy dtype to Julia UInt type
    #     julia_dtype = {np.uint8: 'UInt8', np.uint16: 'UInt16', np.uint32: 'UInt32', np.uint64: 'UInt64'}[dtype]
    #     julia_type = Main.eval(julia_dtype)
        
    #     # Call the Julia function to perform quantization and directly access struct fields
    #     quantized_data = Main.eval(f"LinQuantization({julia_type}, {array.tolist()})")
    #     min_val = Main.eval(f"LinQuantization({julia_type}, {array.tolist()}).min")
    #     max_val = Main.eval(f"LinQuantization({julia_type}, {array.tolist()}).max")
        
    #     # Convert quantized data to bytes and return
    #     return bytes(quantized_data), min_val, max_val

    def decode(self, data: bytes, dtype: np.dtype, shape: tuple) -> np.ndarray:
        """Reconstructs the numpy array from the quantized data."""
        assert dtype in [np.uint8, np.uint16, np.uint32, np.uint64], "dtype must be an unsigned integer type"
        
        # Convert bytes back to numpy array of the appropriate dtype
        quant_array = np.frombuffer(data, dtype=dtype).reshape(shape)
        
        # Mapping numpy dtype to Julia UInt type
        julia_dtype = {np.uint8: 'UInt8', np.uint16: 'UInt16', np.uint32: 'UInt32', np.uint64: 'UInt64'}[dtype]
        
        # Create a fake LinQuantArray structure expected by Julia for dequantization
        # Main.eval(f"""
        # struct LinQuantArray{T,N} <: AbstractArray{{Unsigned,N}}
        #     A::Array{{T,N}}
        #     min::Float64
        #     max::Float64
        # end
        # """)
        julia_type = Main.eval(julia_dtype)

        

        Main.eval(f"""
        struct LinQuantArray{{T, N}} <: AbstractArray{{Unsigned, N}}
            A::Array{{T, N}}
            min::Float64
            max::Float64
        end
        """)

        dummy_min, dummy_max = 0.0, 1.0
        Main.eval(f"q_array = LinQuantArray{{{julia_type}, 1}}(reshape({quant_array.tolist()}, {shape}), {dummy_min}, {dummy_max})")        
        
        
        
        # # Call the dequantization function
        result = Main.Array(julia_type, Main.q_array)
        return result

In [40]:
compressor = LinQuantArrays()
arr = np.array([
    0.5848807, 0.5846372, 0.22117455, 0.875293, 0.13118872, 0.6605228,
    0.51365334, 0.07956224, 0.09270971, 0.424567, 0.3399181, 0.61007863,
    0.6623791, 0.41715723, 0.49276868, 0.29014704
])
print("Original array:", arr)
print("Arr size:", sys.getsizeof(arr))

result,encoded = compressor.encode(arr, dtype=np.uint8)

print("Encoded array:", encoded)
print("Encoded array size:", sys.getsizeof(encoded))


Original array: [0.5848807  0.5846372  0.22117455 0.875293   0.13118872 0.6605228
 0.51365334 0.07956224 0.09270971 0.424567   0.3399181  0.61007863
 0.6623791  0.41715723 0.49276868 0.29014704]
Arr size: 232
Encoded array: b'\xa2\xa2-\xff\x11\xba\x8b\x00\x04oS\xaa\xbbl\x84C'
Encoded array size: 49


In [20]:
result

[162, 162, 45, 255, 17, 186, 139, 0, 4, 111, 83, 170, 187, 108, 132, 67]

In [41]:
decoded = compressor.decode(encoded, dtype=np.uint8, shape=(16,))
print("Decoded array:", decoded)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching Array(::Type{UInt8}, ::Vector{Int64})
Stacktrace:
 [1] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::@Kwargs{})
   @ Base .\essentials.jl:892
 [2] invokelatest(::Any, ::Any, ::Vararg{Any})
   @ Base .\essentials.jl:889
 [3] _pyjlwrap_call(f::Type, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Asus\.julia\packages\PyCall\1gn3u\src\callback.jl:28
 [4] pyjlwrap_call(self_::Ptr{PyCall.PyObject_struct}, args_::Ptr{PyCall.PyObject_struct}, kw_::Ptr{PyCall.PyObject_struct})
   @ PyCall C:\Users\Asus\.julia\packages\PyCall\1gn3u\src\callback.jl:44>

In [16]:
compressor = LinQuantArrays()
arr = np.array([
    0.5848807, 0.5846372, 0.22117455, 0.875293, 0.13118872, 0.6605228,
    0.51365334, 0.07956224, 0.09270971, 0.424567, 0.3399181, 0.61007863,
    0.6623791, 0.41715723, 0.49276868, 0.29014704
])

encoded, min_val, max_val = compressor.encode(arr, dtype=np.uint8)
print("Encoded Data:", encoded)
print("Min Value:", min_val)
print("Max Value:", max_val)

ValueError: too many values to unpack (expected 3)